In [1]:
import folium
import requests
from utils import allowed_gai_family, parse_xml
from calcErr import Point
import urllib3
import json
urllib3.util.connection.allowed_gai_family = allowed_gai_family

filePath ="./trajectories/0.gpx"
kp_dis_threshold = 50
kp_connum_threshold = 3
algorithm = 1 # 0:SP, 1:KMM, 3:ST-MM
pathCalcAlgo = 0  # 0：CH, 1:Dijkstra
pathCacheType = 1  # 0:no cache 1:cache
weightType = 0 # 0:distance 1:time
city = "shanghai"

if algorithm == 0:
    algorithmName = "SP"
elif algorithm == 1:
    algorithmName = "KMM"
elif algorithm == 3:
    algorithmName = "ST-MM"
else:
    raise Exception("algorithm err")

url = 'http://localhost:8999/match?profile=car&type=json&gps_accuracy=20&keypoint_num=999&kp_dis_threshold={}&kp_connum_threshold={}&algorithm={}&useVelocity=true&velocityThreshold=60&path_calc_algo={}&path_cache_type={}&weight_type={}'.format(
    kp_dis_threshold, kp_connum_threshold, algorithm, pathCalcAlgo, pathCacheType,weightType)

with open(filePath, 'rb') as f:
    r = requests.post(url, data=f, headers={'Content-Type': 'application/gpx+xml'})
    if r.status_code == 200:
        j = json.loads(r.text)
        _, x = parse_xml(filePath)
        matchPath = []
        realPath = []
        map = folium.Map(location=[j['latitudes'][0],j['longitudes'][1]], zoom_start=12)
        for i in range(0, len(j['latitudes'])-1):
            matchPath.append(Point(j['latitudes'][i], j['longitudes'][i]))
            folium.PolyLine(locations=[[j['latitudes'][i],j['longitudes'][i]],[j['latitudes'][i+1],j['longitudes'][i+1]]], color='green', weight=5).add_to(map)   
        for i in range(0, len(x)-1):
            realPath.append(Point(x.iloc[i]['lat'], x.iloc[i]['lon']))
            folium.PolyLine(locations=[[x.iloc[i]['lat'],x.iloc[i]['lon']],[x.iloc[i+1]['lat'],x.iloc[i+1]['lon']]], color='blue', weight=2).add_to(map)
map